In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import os


In [2]:
raw_text = ''
dataset_folder = 'text-data'

In [3]:
def read_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read();

In [4]:
for file in os.listdir(dataset_folder):
    raw_text = raw_text + read_file(dataset_folder+'/'+file)

In [5]:
raw_text = raw_text.lower()

In [6]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  738582
Total Vocab:  102


In [7]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  738482


In [8]:
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)
print(X.shape, y.shape)

torch.Size([738482, 100, 1]) torch.Size([738482])


In [9]:
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x

In [ ]:
n_epochs = 40
batch_size = 128
model = CharModel()

optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=batch_size)

best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss += loss_fn(y_pred, y_batch)
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))

torch.save([best_model, char_to_dict], "single-char.pth")

Epoch 0: Cross-entropy: 2075231.5000
Epoch 1: Cross-entropy: 2021502.3750
Epoch 2: Cross-entropy: 1969324.3750
Epoch 3: Cross-entropy: 1924242.5000
Epoch 4: Cross-entropy: 1873051.7500
Epoch 5: Cross-entropy: 1825994.3750
Epoch 6: Cross-entropy: 1786455.2500
Epoch 7: Cross-entropy: 1776091.7500
Epoch 8: Cross-entropy: 1730006.5000
Epoch 9: Cross-entropy: 1707139.2500
Epoch 10: Cross-entropy: 1686052.2500
Epoch 11: Cross-entropy: 1671037.1250
Epoch 12: Cross-entropy: 1656166.0000
Epoch 13: Cross-entropy: 1637354.6250
Epoch 14: Cross-entropy: 1625378.8750
Epoch 15: Cross-entropy: 1611737.1250
Epoch 16: Cross-entropy: 1593314.0000
Epoch 17: Cross-entropy: 1590881.0000
Epoch 18: Cross-entropy: 1578689.5000
Epoch 19: Cross-entropy: 1568103.1250
Epoch 20: Cross-entropy: 1558731.3750
Epoch 21: Cross-entropy: 1552494.6250
Epoch 22: Cross-entropy: 1547819.8750
Epoch 23: Cross-entropy: 1537509.5000
Epoch 24: Cross-entropy: 1540575.8750
Epoch 25: Cross-entropy: 1533812.2500
Epoch 26: Cross-entrop